In [7]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/dhruvsaluja/indian-medicinal-plants")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dhruvsaluja
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/dhruvsaluja/indian-medicinal-plants


100%|██████████| 253M/253M [00:04<00:00, 54.1MB/s]


In [4]:
# Define parameters
input_shape = (150, 150, 3)  # Adjust the image size as needed
batch_size = 32
epochs = 20
num_classes = 40  # Number of different plant classes
data_dir = "/content/indian-medicinal-plants/Indian Medicinal Plant Image Datasets/Medicinal plant dataset"  # Replace with the path to your dataset folder


In [5]:
# Create a mapping between class indices and plant names
class_mapping = {
    0: 'Aloevera',
    1: 'Amla',
    2: 'Amruta Balli',
    3: 'Arali',
    4: 'Ashoka',
    5: 'Ashwagandha',
    6: 'Avacado',
    7: 'Bamboo',
    8: 'Basale',
    9: 'Betel',
    10: 'Betel_Nut',
    11: 'Brahmi',
    12: 'Castor',
    13: 'Curry Leaf',
    14: 'Doddapatre',
    15: 'Ekka',
    16: 'Ganike',
    17: 'Gauva',
    18: 'Geranium',
    19: 'Henna',
    20: 'Hibiscus',
    21: 'Honge',
    22: 'Insulin',
    23: 'Jasmine',
    24: 'Lemon',
    25: 'Lemon_grass',
    26: 'Mango',
    27: 'Mint',
    28: 'Nagadali',
    29: 'Neem',
    30: 'Nithyapushpa',
    31: 'Nooni',
    32: 'Pappaya',
    33: 'Pepper',
    34: 'Pomegranate',
    35: 'Raktachandini',
    36: 'Rose',
    37: 'Sapota',
    38: 'Tulasi',
    39: 'Wood_sorel',
    # Add mappings for all 40 classes here
}

In [8]:
# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,     # Normalize pixel values
    rotation_range=20,      # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,        # Shear intensity
    zoom_range=0.2,         # Randomly zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest',    # Fill missing pixels with the nearest value
    validation_split=0.2     # 20% of data will be used for validation
)

In [9]:
# Load and augment training data (80%)
train_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],   # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=True,             # Shuffle the data for training
    subset='training'         # Specify training data subset
)

Found 4765 images belonging to 40 classes.


In [10]:
# Load and augment validation data (20%)
validation_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],  # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False,            # Do not shuffle for validation
    subset='validation'       # Specify validation data subset
)


Found 1180 images belonging to 40 classes.


In [ ]:
# Define input shape (modify this according to your input data)
input_shape = (150, 150, 3)  # Assuming input images are 224x224 RGB images
num_classes = 40  # Define the number of classes in your dataset


In [13]:

# Load the trained models for inference
loaded_model_201 = tf.keras.models.load_model('plant_classification_model_DenseNet121.h5')
loaded_model_169 = tf.keras.models.load_model('plant_classification_model_DenseNet169.h5')
loaded_model_121 = tf.keras.models.load_model('plant_classification_model_DenseNet201.h5')


In [14]:
# Initialize lists to store true labels and predicted labels
y_true = []
y_pred = []

In [15]:
# Loop through the validation generator and make predictions
for i in range(len(validation_generator)):
    # Get the batch of images and labels
    x_batch, y_batch = validation_generator[i]

    # Make predictions with the ensemble model
    predictions_201 = loaded_model_201.predict(x_batch)
    predictions_169 = loaded_model_169.predict(x_batch)
    predictions_121 = loaded_model_121.predict(x_batch)

    # Combine predictions (simple averaging)
    ensemble_predictions = (predictions_201 + predictions_169 + predictions_121) / 3.0

    # Get the predicted class indices
    predicted_indices = np.argmax(ensemble_predictions, axis=1)

    # Append true labels and predicted labels to the lists
    y_true.extend(np.argmax(y_batch, axis=1))
    y_pred.extend(predicted_indices)

1/1 [==============================] - 8s 8s/step


In [16]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

In [17]:
# Print evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.8966101694915254
Precision: 0.8998235533328679
Recall: 0.8966101694915254
F1 Score: 0.894781519791972


In [18]:

# Define a function for ensemble prediction
def predict_plant_ensemble(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # Predict with each model
    prediction_201 = loaded_model_201.predict(img_array)
    prediction_169 = loaded_model_169.predict(img_array)
    prediction_121 = loaded_model_121.predict(img_array)

    # Combine predictions (e.g., simple averaging)
    ensemble_prediction = (prediction_201 + prediction_169 + prediction_121) / 3.0

    # Get the predicted class index
    class_index = np.argmax(ensemble_prediction)

    # Map the class index to plant name
    plant_name = class_mapping.get(class_index, 'Unknown Plant')

    # Get the confidence level
    confidence = ensemble_prediction[0, class_index]

    return plant_name, confidence



In [24]:
# Example usage:
image_path = '/content/test4.jpg'
predicted_plant, confidence = predict_plant_ensemble(image_path)
print(f"Predicted plant: {predicted_plant}")
print(f"Confidence level: {confidence * 100:.2f}%")

1/1 [==============================] - 0s 216ms/step
Predicted plant: Hibiscus
Confidence level: 88.18%
